In [ ]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import json
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.keypoint_utils.body_parts import BodyParts
from research.utils.image_utils import Picture
from research.weight_estimation.keypoint_utils.akpr import get_homography_and_matches, generate_sift_adjustment
from research.weight_estimation.keypoint_utils.optics import pixel2world

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
body_parts = BodyParts().get_core_body_parts()
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM keypoint_annotations
    WHERE pen_id=5
    AND captured_at BETWEEN '2019-06-05' AND '2019-07-02'
    AND keypoints is not null
    AND keypoints -> 'leftCrop' is not null
    AND keypoints -> 'rightCrop' is not null
    AND is_qa = TRUE;
"""

mdf = rds_access_utils.extract_from_database(query)
print('Manual dataframe loaded!')

adf = pd.concat([
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-05,2019-06-12).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-12,2019-06-19).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-19,2019-06-26).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-26,2019-07-03).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-07-03,2019-07-04).csv')
])

adf = adf[adf.akpd_score > 0.9].copy(deep=True)
print('AKPD dataframe loaded!')

url_intersection = sorted(list(set(mdf.left_image_url).intersection(adf.left_crop_url)))
df = adf[adf.left_crop_url.isin(url_intersection)].sort_values('left_crop_url')
df['manual_keypoints'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').keypoints.values
df['camera_metadata'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').camera_metadata.values
df['left_crop_metadata'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').left_crop_metadata.values
df['right_crop_metadata'] = mdf[mdf.left_image_url.isin(url_intersection)].sort_values('left_image_url').right_crop_metadata.values



df = df.sort_values('captured_at')
df['estimated_weight_g'] = df.weight
df.index = pd.to_datetime(df.captured_at)
print('Manual and AKPD Dataframes Joined')

In [ ]:
count = 0
keypoint_data, analysis_data = defaultdict(list), defaultdict(list)
adj_anns = []
for idx, row in df.head(1000).iterrows():
    print(count)
    count += 1
    
    # get image information and metadata
    left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
    
    # get keypoint coordinates
    ann = row.manual_keypoints
    left_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in ann['leftCrop']}
    right_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in ann['rightCrop']}
    left_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in ann['leftCrop']}
    right_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in ann['rightCrop']}
    
    # jitter keypoints
    akpd_ann = json.loads(row.annotation)
    jittered_left_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in akpd_ann['leftCrop']}
    jittered_right_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in akpd_ann['rightCrop']}
    jittered_left_kps_frame = {bp: jittered_left_kps[bp] + np.array([left_crop_metadata['x_coord'], left_crop_metadata['y_coord']]) 
                               for bp in body_parts}
    jittered_right_kps_frame = {bp: jittered_right_kps[bp] + np.array([right_crop_metadata['x_coord'], right_crop_metadata['y_coord']]) 
                               for bp in body_parts}
    

    left_fish_picture = Picture(s3_access_utils=s3_access_utils, image_url=left_crop_url)
    right_fish_picture = Picture(s3_access_utils=s3_access_utils, image_url=right_crop_url)
    left_fish_picture.enhance(in_place=True, sharpen=True)
    right_fish_picture.enhance(in_place=True, sharpen=True)
    sift = cv2.KAZE_create()
    left_items, right_items = [], []
    for bp in body_parts:
        
        try:
            left_item, right_item, num_matches = generate_sift_adjustment(bp, left_crop_metadata, left_fish_picture,
                                                                          jittered_left_kps, right_crop_metadata,
                                                                          right_fish_picture, jittered_right_kps, sift)
        except:
            continue
        left_items.append(left_item)
        right_items.append(right_item)
        
        original_disp = abs(left_kps_frame[bp][0] - right_kps_frame[bp][0])
        jittered_disp = abs(jittered_left_kps_frame[bp][0] - jittered_right_kps_frame[bp][0])
        adj_disp = abs(left_item['xFrame'] - right_item['xFrame'])
        
        analysis_data['left_crop_url'].append(left_crop_url)
        analysis_data['body_part'].append(bp)
        analysis_data['original_disp'].append(original_disp)
        analysis_data['jittered_disp'].append(jittered_disp)
        analysis_data['adj_disp'].append(adj_disp)
        analysis_data['num_matches'].append(num_matches)
    
    adj_ann = {
        'leftCrop': left_items,
        'rightCrop': right_items
    }
    keypoint_data['left_crop_url'].append(left_crop_url)
    keypoint_data['right_crop_url'].append(right_crop_url)
    keypoint_data['ann'].append(ann)
    keypoint_data['akpd_ann'].append(akpd_ann)
    keypoint_data['adj_ann'].append(adj_ann)
    
    

In [ ]:
analysis_df = pd.DataFrame(analysis_data)
keypoint_df = pd.DataFrame(keypoint_data)

    

In [ ]:
left_fish_picture.get_image()

In [ ]:
analysis_df['abs_error'] = (analysis_df.adj_disp - analysis_df.original_disp).abs()
analysis_df[(analysis_df.num_matches > 30) & (analysis_df.abs_error > 50)]

In [ ]:
analysis_df.shape

In [ ]:
left_crop_url = 'https://s3-eu-west-1.amazonaws.com/aquabyte-crops/environment=production/site-id=23/pen-id=5/date=2019-06-06/hour=06/at=2019-06-06T06:13:57.247786000Z/left_frame_crop_1430_366_3322_1227.jpg'
row = keypoint_df[keypoint_df.left_crop_url == left_crop_url].iloc[0]
left_crop_url, right_crop_url, ann, adj_ann = row.left_crop_url, row.right_crop_url, row.ann, row.adj_ann
akpd_ann = json.loads(df[df.left_crop_url == left_crop_url].annotation.iloc[0])
left_crop_metadata = df[df.left_crop_url == left_crop_url].left_crop_metadata.iloc[0]
right_crop_metadata = df[df.left_crop_url == left_crop_url].right_crop_metadata.iloc[0]
left_picture, right_picture = Picture(s3_access_utils=s3_access_utils, image_url=left_crop_url), \
                              Picture(s3_access_utils=s3_access_utils, image_url=right_crop_url)
left_picture.enhance(sharpen=False)
right_picture.enhance(sharpen=False)
###

left_image_arr = left_picture.get_image_arr()
blurred = cv2.GaussianBlur(left_image_arr, (21, 21), 0)
sharpened = cv2.addWeighted(left_image_arr, 2.0, blurred, -1.0, 0)
left_picture.image_arr = sharpened
left_picture.image = Image.fromarray(sharpened)

right_image_arr = right_picture.get_image_arr()
blurred = cv2.GaussianBlur(right_image_arr, (21, 21), 0)
sharpened = cv2.addWeighted(right_image_arr, 2.0, blurred, -1.0, 0)
right_picture.image_arr = sharpened
right_picture.image = Image.fromarray(sharpened)

###

left_crop_image, right_crop_image = left_picture.get_image(), right_picture.get_image()
left_draw, right_draw = ImageDraw.Draw(left_crop_image), ImageDraw.Draw(right_crop_image)

r = 3
for item in akpd_ann['leftCrop']:
    x, y = item['xCrop'], item['yCrop']
    left_draw.ellipse((x - r, y - r, x + r, y + r), fill='red', outline='red')
for item in akpd_ann['rightCrop']:
    x, y = item['xCrop'], item['yCrop']
    right_draw.ellipse((x - r, y - r, x + r, y + r), fill='red', outline='red')

for item in adj_ann['leftCrop']:
    x, y = item['xCrop'], item['yCrop']
    left_draw.ellipse((x - r, y - r, x + r, y + r), fill='green', outline='green')
for item in adj_ann['rightCrop']:
    x, y = item['xCrop'], item['yCrop']
    right_draw.ellipse((x - r, y - r, x + r, y + r), fill='green', outline='green')

    
left_crop_image


In [ ]:
right_crop_image

In [ ]:
from scipy.spatial import Delaunay
from itertools import compress

def in_hull(p, hull):
    hull = Delaunay(hull)
    return hull.find_simplex(p)>=0


def apply_convex_hull_filter(kp, des, canonical_kps, bbox):
    X_canon_kps = np.array(list(canonical_kps.values()))
    X_kp = np.array([x.pt for x in kp]).reshape(-1, 2) + np.array([bbox['x_min'], bbox['y_min']])
    is_valid = in_hull(X_kp, X_canon_kps)
    kp = list(compress(kp, is_valid))
    des = des[is_valid]
    return kp, des


def get_homography_and_matches(sift, left_patch, right_patch,
                               left_kps, right_kps,
                               left_bbox, right_bbox,
                               good_perc=0.7, min_match_count=3):

    kp1, des1 = sift.detectAndCompute(left_patch, None)
    kp2, des2 = sift.detectAndCompute(right_patch, None)

#     apply convex hull filter
    kp1, des1 = apply_convex_hull_filter(kp1, des1, left_kps, left_bbox)
    kp2, des2 = apply_convex_hull_filter(kp2, des2, right_kps, right_bbox)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < good_perc * n.distance:
            good.append(m)

    H, matches_mask = np.eye(3), None
    if len(good) >= min_match_count:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()
    return H, kp1, kp2, good, matches_mask

In [ ]:
sift = cv2.KAZE_create()
left_items, right_items = [], []
bp = 'UPPER_LIP'

left_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in akpd_ann['leftCrop']}
right_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in akpd_ann['rightCrop']}
left_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in akpd_ann['leftCrop']}
right_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in akpd_ann['rightCrop']}

left_kp, right_kp = left_kps[bp], right_kps[bp]
left_crop, left_bbox = left_picture.generate_crop_given_center(left_kp[0], left_kp[1], 600, 200)
right_crop, right_bbox = right_picture.generate_crop_given_center(right_kp[0], right_kp[1], 600, 200)
H, kp1, kp2, good, matches_mask = get_homography_and_matches(sift, left_crop, right_crop, 
                                                            left_kps, right_kps,
                                                            left_bbox, right_bbox)
plt.figure(figsize=(20, 10))

img1 = np.array(left_crop)
img2 = np.array(right_crop)

h,w,d = img1.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts, H)
img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

img = cv2.drawMatches(img1,kp1,img2,kp2,good,None,
                      matchesMask=matches_mask, singlePointColor=None)
plt.imshow(img)
plt.show()


In [ ]:
local_left_kp = [left_kp[0] - left_bbox['x_min'], left_kp[1] - left_bbox['y_min']]
local_right_kp = cv2.perspectiveTransform(np.array([local_left_kp[0], local_left_kp[1]]).reshape(-1, 1, 2).astype(float), H).squeeze()
right_kp = [local_right_kp[0] + right_bbox['x_min'], local_right_kp[1] + right_bbox['y_min']]

In [ ]:
left_kp_frame = left_kp + np.array([left_crop_metadata['x_coord'], left_crop_metadata['y_coord']])  
right_kp_frame = right_kp + np.array([right_crop_metadata['x_coord'], right_crop_metadata['y_coord']])
print(left_kp_frame - right_kp_frame)

In [ ]:
sum(matches_mask)

In [ ]:
sum(matches_mask)

In [ ]:
src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

In [ ]:
src_pts.reshape(-1, 2)[np.array(matches_mask).astype(bool)] - \
dst_pts.reshape(-1, 2)[np.array(matches_mask).astype(bool)]

In [ ]:
X = src_pts.reshape(-1, 2)[np.array(matches_mask).astype(bool)]
X = np.hstack([X, np.ones(X.shape[0]).reshape(-1, 1)])

In [ ]:
X_prime = np.dot(H, X.T).T
X_prime[:, 0] = X_prime[:, 0] / X_prime[:, 2]
X_prime[:, 1] = X_prime[:, 1] / X_prime[:, 2]
X_prime[:, 2] = 1.0

In [ ]:
X[:, :-1] - X_prime[:, :-1]

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 15))
for idx, bp in enumerate(body_parts):
    
    row = idx // 3
    col = idx % 3
    
    bp_mask = analysis_df.body_part == bp
    behavior_mask = (analysis_df.num_matches > 20) & (abs(analysis_df.original_disp - analysis_df.adj_disp) < 100)
    pre_adj_err = (analysis_df[bp_mask].jittered_disp - analysis_df[bp_mask].original_disp).values
    post_adj_err = (analysis_df[bp_mask & behavior_mask].adj_disp - analysis_df[bp_mask & behavior_mask].original_disp).values
    
    print(analysis_df[bp_mask & behavior_mask].shape[0] / analysis_df[bp_mask].shape[0])
    
    pre_adj_rms = np.mean(pre_adj_err**2)**0.5
    post_adj_rms = np.mean(post_adj_err**2)**0.5
    
    ax = axes[row, col]
    ax.hist(pre_adj_err, bins=50, color='blue', alpha=0.7)
    ax.hist(post_adj_err, bins=50, color='red', alpha=0.7)
    ax.grid()
    ax.set_title('Body part: {}, Pre RMS: {}, Post RMS: {}'.format(bp, round(pre_adj_rms, 2), 
                                                               round(post_adj_rms, 2)))
plt.show()

In [ ]:
def get_disparity_and_depth(manual_ann, akpd_ann, camera_metadata):
    if 'leftCrop' in manual_ann and 'rightCrop' in manual_ann:
        world_keypoints = pixel2world(manual_ann['leftCrop'], manual_ann['rightCrop'], camera_metadata)
        depth = np.mean([x[1] for x in world_keypoints.values()])
        
        manual_left_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in manual_ann['leftCrop']}
        manual_right_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in manual_ann['rightCrop']}
        akpd_left_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in akpd_ann['leftCrop']}
        akpd_right_keypoints = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in akpd_ann['rightCrop']}
        
        manual_disps = []
        akpd_disps = []
        for bp in body_parts:
            manual_disp = abs(manual_left_keypoints[bp][0] - manual_right_keypoints[bp][0])
            akpd_disp = abs(akpd_left_keypoints[bp][0] - akpd_right_keypoints[bp][0])
            manual_disps.append(manual_disp)
            akpd_disps.append(akpd_disp)
            
        return manual_disps, akpd_disps, [depth] * len(manual_disps)
            
            
        
    
    
    

In [ ]:
all_manual_disps, all_akpd_disps, all_depths = [], [], []
count = 0
for idx, row in df.iterrows():
    if count % 10 == 0:
        print(count)
    count += 1
    manual_ann = row.manual_keypoints
    akpd_ann = json.loads(row.annotation)
    camera_metadata = row.camera_metadata
    manual_disps, akpd_disps, depths = get_disparity_and_depth(manual_ann, akpd_ann, camera_metadata)
    all_manual_disps.extend(manual_disps)
    all_akpd_disps.extend(akpd_disps)
    all_depths.extend(depths)

In [ ]:
tdf = pd.DataFrame({'manual_disp': all_manual_disps, 'akpd_disp': all_akpd_disps, 'depth': all_depths})

In [ ]:
tdf['error'] = tdf.akpd_disp - tdf.manual_disp

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(tdf.depth, tdf.error)
plt.grid()
plt.show()

In [ ]:
low = 2.2
tdf[(tdf.depth > low) & (tdf.depth < low + 0.1)].error.std()